# Problem 2

In [13]:
from __future__ import print_function
import time
import numpy as np

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F

In [15]:
import torchvision
import torchvision.transforms

In [16]:
import matplotlib.pyplot as plt

In [17]:
mnist_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
mnist_train = torchvision.datasets.MNIST(root='./data', train=True, transform=mnist_transforms, download=True)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, transform=mnist_transforms, download=True)

In [37]:
(mnist_train[0][0][0]).shape

torch.Size([28, 28])

In [18]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=64, shuffle=True, num_workers=2)

In [19]:
class Classifier(nn.Module):
    """Convnet Classifier"""
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        # Logistic Regression
        self.clf = nn.Linear(128, 10)

    def forward(self, x):
        return self.clf(self.conv(x).squeeze())


In [20]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


In [21]:
clf = Classifier()
if cuda_available:
    clf = clf.cuda()
optimizer = torch.optim.Adam(clf.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
# This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class.

In [23]:
for epoch in range(50):
    losses = []
    # Train
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        if cuda_available:
            inputs, targets = inputs.cuda(), targets.cuda()

        optimizer.zero_grad()
        outputs = clf(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.item())
        
        if batch_idx%50==0:
            print('Epoch : %d Loss : %.3f ' % (epoch, np.mean(losses)))
    
    # Evaluate
    clf.eval()
    total = 0
    correct = 0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        if cuda_available:
            inputs, targets = inputs.cuda(), targets.cuda()

        outputs = clf(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    print('Epoch : %d Test Acc : %.3f' % (epoch, 100.*correct/total))
    print('--------------------------------------------------------------')
    clf.train()

Epoch : 0 Loss : 1.428 
Epoch : 0 Loss : 1.265 
Epoch : 0 Loss : 1.075 
Epoch : 0 Loss : 0.943 
Epoch : 0 Loss : 0.850 
Epoch : 0 Loss : 0.778 
Epoch : 0 Loss : 0.723 
Epoch : 0 Loss : 0.676 
Epoch : 0 Loss : 0.637 
Epoch : 0 Loss : 0.604 
Epoch : 0 Loss : 0.574 
Epoch : 0 Loss : 0.550 
Epoch : 0 Loss : 0.527 
Epoch : 0 Loss : 0.508 
Epoch : 0 Loss : 0.489 
Epoch : 0 Loss : 0.474 
Epoch : 0 Loss : 0.460 
Epoch : 0 Loss : 0.446 
Epoch : 0 Loss : 0.434 
Epoch : 0 Test Acc : 93.000
--------------------------------------------------------------
Epoch : 1 Loss : 2.422 
Epoch : 1 Loss : 1.399 
Epoch : 1 Loss : 0.968 
Epoch : 1 Loss : 0.797 
Epoch : 1 Loss : 0.695 
Epoch : 1 Loss : 0.639 
Epoch : 1 Loss : 0.596 
Epoch : 1 Loss : 0.568 
Epoch : 1 Loss : 0.545 
Epoch : 1 Loss : 0.526 
Epoch : 1 Loss : 0.510 
Epoch : 1 Loss : 0.494 
Epoch : 1 Loss : 0.482 
Epoch : 1 Loss : 0.471 
Epoch : 1 Loss : 0.461 
Epoch : 1 Loss : 0.451 
Epoch : 1 Loss : 0.441 
Epoch : 1 Loss : 0.434 
Epoch : 1 Loss : 0.42